# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [102]:
#!pip install selenium

In [103]:
## Webdriver
# Firefox -> geckodriver
# Chrome -> chromedriver -> pesquisar chromedriver download

In [104]:
# Passo a passo

from selenium import webdriver

# Passo 1: Abrir o navegador
navegador = webdriver.Chrome()
navegador.get("https://google.com")

In [105]:
# Passo 2: Importar a base de dados
import pandas as pd

tabela = pd.read_excel("commodities.xlsx")
display(tabela)           

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


In [106]:
# Passo 3: Para cada produto na base de dados, fazer:
import unidecode

for linha in tabela.index:
    produto_tabela = tabela.loc[linha, "Produto"]
    produto = unidecode.unidecode(produto_tabela) # Decodifica para tirar caracteres especiais
    
    link = f"https://www.melhorcambio.com/{produto}-hoje"
    navegador.get(link)
    
    #.click() -> clicar
    #.send_keys("texto") -> escrever
    #.get_attribute() -> pegar um valor    
    
    # Passo 4: Pesquisar o preço do produto    
    preco_produto = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    #Formatar o preço para padrão americano
    preco_produto = preco_produto.replace(".", "").replace(",", ".")

    # Passo 5: Atualizar o preço na base de dados
    tabela.loc[linha, "Preço Atual"] = float(preco_produto)

display(tabela)


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.24,NaN
1,Soja,163.59,160.78,NaN
2,Boi,282.20,278.90,NaN
3,Petróleo,424.37,382.52,NaN
4,Algodão,497.76,480.00,NaN
5,Açúcar,136.23,135.68,NaN
6,Café,1092.87,1113.46,NaN
7,Ouro,321.77,334.72,NaN
8,Trigo,1549.11,1548.70,NaN
9,Tilápia,9.05,9.05,NaN


In [107]:
# Passo 6: Decidir quais produtos comprar
#Preencher a coluna de comprar

tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"]

display(tabela)

#Exportar base para o Excel
tabela.to_excel("commoditiesAtualizado.xlsx", index = False)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.24,True
1,Soja,163.59,160.78,True
2,Boi,282.20,278.90,True
3,Petróleo,424.37,382.52,True
4,Algodão,497.76,480.00,True
5,Açúcar,136.23,135.68,True
6,Café,1092.87,1113.46,False
7,Ouro,321.77,334.72,False
8,Trigo,1549.11,1548.70,True
9,Tilápia,9.05,9.05,False


In [108]:
navegador.quit()